## Implementing binary decision trees

In [99]:
# Import the Classifier & required libraries
import pandas as pd
import numpy as np
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

In [100]:
# Load the lending club dataset
loans = pd.read_csv('lending-club-data.csv')

In [101]:
# Create variable for Safe Loan
loans['safe_loans'] = loans['bad_loans'].apply(lambda x : +1 if x==0 else -1)
loans = loans.drop(columns = ['bad_loans'])

Unlike the previous assignment where we used several features, in this assignment, we will just be using 4 categorical
features: 

1. grade of the loan 
2. the length of the loan term
3. the home ownership status: own, mortgage, rent
4. number of years of employment.

Since we are building a binary decision tree, we will have to convert these categorical features to a binary representation in a subsequent section using 1-hot encoding.

In [104]:

features = ['grade',              # grade of the loan
            'term',               # the term of the loan
            'home_ownership',     # home_ownership status: own, mortgage or rent
            'emp_length',         # number of years of employment
           ]
target = 'safe_loans'
loans = loans[features + [target]]


# Modeling only works well on clean data
loans = loans.dropna()

In [107]:
# Let's explore what the dataset looks like.
loans

,grade,term,home_ownership,emp_length,safe_loans
0,E,60 months,MORTGAGE,3 years,1
1,C,36 months,RENT,10+ years,1
2,E,60 months,MORTGAGE,10+ years,-1
3,D,60 months,RENT,7 years,1
4,E,36 months,MORTGAGE,10+ years,1
...,...,...,...,...,...
4725,B,36 months,MORTGAGE,4 years,-1
4727,D,36 months,MORTGAGE,10+ years,1
4728,D,60 months,MORTGAGE,5 years,-1
4729,D,60 months,MORTGAGE,10+ years,-1


## Transform categorical data into binary features

In this assignment, we will implement **binary decision trees** (decision trees for binary features, a specific case of categorical variables taking on two values, e.g., true/false). Since all of our features are currently categorical features, we want to turn them into binary features. 

For instance, the **home_ownership** feature represents the home ownership status of the loanee, which is either `own`, `mortgage` or `rent`. For example, if a data point has the feature 
```
   {'home_ownership': 'RENT'}
```
we want to turn this into three features: 
```
 { 
   'home_ownership = OWN'      : 0, 
   'home_ownership = MORTGAGE' : 0, 
   'home_ownership = RENT'     : 1
 }
```

Since this code requires a few Python and GraphLab tricks, feel free to use this block of code as is. Refer to the API documentation for a deeper understanding.

In [108]:
loans.loc[loans['home_ownership'] != 'RENT', 'home_ownership'] = 0
loans.loc[loans['home_ownership'] == 'RENT', 'home_ownership'] = 1

Let's see what the feature columns look like now:

In [135]:
features = loans.drop(columns = ['safe_loans'])  # Remove the response variable
features_list = list(features.columns)
features_list

['grade', 'term', 'home_ownership', 'emp_length']

In [110]:
features

,grade,term,home_ownership,emp_length
0,E,60 months,0,3 years
1,C,36 months,1,10+ years
2,E,60 months,0,10+ years
3,D,60 months,1,7 years
4,E,36 months,0,10+ years
...,...,...,...,...
4725,B,36 months,0,4 years
4727,D,36 months,0,10+ years
4728,D,60 months,0,5 years
4729,D,60 months,0,10+ years


In [150]:
# Initiate OneHot Encoding of the categories

# Define the encoder
encoder = OneHotEncoder()

# Encode the existing variables
encoded_values = encoder.fit_transform(features)
#decode the variables into an array
decoded_matrix = encoded_values.toarray()
# get names for the decoded matrix
encoded_feature_names = encoder.get_feature_names()
# Create Data Frame from the 
features = pd.DataFrame(decoded_matrix, columns=encoded_feature_names)

,x0_A,x0_B,x0_C,x0_D,x0_E,x0_F,x0_G,x1_ 36 months,x1_ 60 months,x2_0,...,x3_10+ years,x3_2 years,x3_3 years,x3_4 years,x3_5 years,x3_6 years,x3_7 years,x3_8 years,x3_9 years,x3_< 1 year
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4488,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4489,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4490,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4491,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [132]:
encoded_values.toarray().shape

(4493, 22)

In [78]:
print("Number of features (after binarizing categorical variables) = %s" % len(features.columns))

Number of features (after binarizing categorical variables) = 4


In [79]:
loans.head(n=1)

,grade,term,home_ownership,emp_length,safe_loans
0,E,60 months,0,3 years,1


Let's explore what one of these columns looks like:

In [80]:
grade_a = loans[loans['grade'] == 'A']
grade_a

,grade,term,home_ownership,emp_length,safe_loans
34,A,36 months,0,3 years,1
40,A,36 months,1,4 years,1
42,A,36 months,0,2 years,1
44,A,36 months,0,10+ years,1
69,A,36 months,0,4 years,1
...,...,...,...,...,...
4694,A,36 months,0,6 years,-1
4701,A,36 months,1,10+ years,1
4712,A,36 months,0,3 years,1
4722,A,36 months,0,2 years,1


This column is set to 1 if the loan grade is A and 0 otherwise.

**Checkpoint:** Make sure the following answers match up.

In [81]:
print("Total number of grade.A loans : %s" % grade_a['safe_loans'].sum())
print("Expected answer               : 6422")

Total number of grade.A loans : 517
Expected answer               : 6422


## Train-test split

We split the data into a train test split with 80% of the data in the training set and 20% of the data in the test set. We use `seed=1` so that everyone gets the same result.

In [87]:
train_data, test_data = train_test_split(loans, test_size = 0.5, random_state = 1)
x_train = train_data.drop(columns = ['safe_loans'])
y_train = train_data['safe_loans']

x_test = test_data.drop(columns = ['safe_loans'])
y_test = test_data['safe_loans']

# Decision tree implementation

In this section, we will implement binary decision trees from scratch. There are several steps involved in building a decision tree. For that reason, we have split the entire assignment into several sections.

## Function to count number of mistakes while predicting majority class

Recall from the lecture that prediction at an intermediate node works by predicting the **majority class** for all data points that belong to this node.

Now, we will write a function that calculates the number of **missclassified examples** when predicting the **majority class**. This will be used to help determine which feature is the best to split on at a given node of the tree.

**Note**: Keep in mind that in order to compute the number of mistakes for a majority classifier, we only need the label (y values) of the data points in the node. 

** Steps to follow **:
* ** Step 1:** Calculate the number of safe loans and risky loans.
* ** Step 2:** Since we are assuming majority class prediction, all the data points that are **not** in the majority class are considered **mistakes**.
* ** Step 3:** Return the number of **mistakes**.


Now, let us write the function `intermediate_node_num_mistakes` which computes the number of misclassified examples of an intermediate node given the set of labels (y values) of the data points contained in the node. Fill in the places where you find `## YOUR CODE HERE`. There are **three** places in this function for you to fill in.

In [88]:
binary_decision_tree = tree.DecisionTreeClassifier()
binary_decision_tree.fit(x_train, y_train)

ValueError: could not convert string to float: 'C'

Because there are several steps in this assignment, we have introduced some stopping points where you can check your code and make sure it is correct before proceeding. To test your `intermediate_node_num_mistakes` function, run the following code until you get a **Test passed!**, then you should proceed. Otherwise, you should spend some time figuring out where things went wrong.

In [69]:
# Test case 1
example_labels = turicreate.SArray([-1, -1, 1, 1, 1])
if intermediate_node_num_mistakes(example_labels) == 2:
    print('Test passed!')
else:
    print('Test 1 failed... try again!')

# Test case 2
example_labels = turicreate.SArray([-1, -1, 1, 1, 1, 1, 1])
if intermediate_node_num_mistakes(example_labels) == 2:
    print('Test passed!')
else:
    print('Test 2 failed... try again!')
    
# Test case 3
example_labels = turicreate.SArray([-1, -1, -1, -1, -1, 1, 1])
if intermediate_node_num_mistakes(example_labels) == 2:
    print('Test passed!')
else:
    print('Test 3 failed... try again!')

NameError: name 'turicreate' is not defined

## Function to pick best feature to split on

The function **best_splitting_feature** takes 3 arguments: 
1. The data (SFrame of data which includes all of the feature columns and label column)
2. The features to consider for splits (a list of strings of column names to consider for splits)
3. The name of the target/label column (string)

The function will loop through the list of possible features, and consider splitting on each of them. It will calculate the classification error of each split and return the feature that had the smallest classification error when split on.

Recall that the **classification error** is defined as follows:
$$
\mbox{classification error} = \frac{\mbox{# mistakes}}{\mbox{# total examples}}
$$

Follow these steps: 
* **Step 1:** Loop over each feature in the feature list
* **Step 2:** Within the loop, split the data into two groups: one group where all of the data has feature value 0 or False (we will call this the **left** split), and one group where all of the data has feature value 1 or True (we will call this the **right** split). Make sure the **left** split corresponds with 0 and the **right** split corresponds with 1 to ensure your implementation fits with our implementation of the tree building process.
* **Step 3:** Calculate the number of misclassified examples in both groups of data and use the above formula to compute the **classification error**.
* **Step 4:** If the computed error is smaller than the best error found so far, store this **feature and its error**.

This may seem like a lot, but we have provided pseudocode in the comments in order to help you implement the function correctly.

**Note:** Remember that since we are only dealing with binary features, we do not have to consider thresholds for real-valued features. This makes the implementation of this function much easier.

Fill in the places where you find `## YOUR CODE HERE`. There are **five** places in this function for you to fill in.

In [29]:
def best_splitting_feature(data, features, target):
    
    best_feature = None # Keep track of the best feature 
    best_error = 10     # Keep track of the best error so far 
    # Note: Since error is always <= 1, we should intialize it with something larger than 1.

    # Convert to float to make sure error gets computed correctly.
    num_data_points = float(len(data))  
    
    # Loop through each feature to consider splitting on that feature
    for feature in features:
        
        # The left split will have all data points where the feature value is 0
        left_split = data[data[feature] == 0]
        
        # The right split will have all data points where the feature value is 1
        ## YOUR CODE HERE
        right_split =  data[data[feature] == 1]
            
        # Calculate the number of misclassified examples in the left split.
        # Remember that we implemented a function for this! (It was called intermediate_node_num_mistakes)
        # YOUR CODE HERE
        left_mistakes = intermediate_node_num_mistakes(left_split[target])            

        # Calculate the number of misclassified examples in the right split.
        ## YOUR CODE HERE
        right_mistakes = intermediate_node_num_mistakes(right_split[target])
            
        # Compute the classification error of this split.
        # Error = (# of mistakes (left) + # of mistakes (right)) / (# of data points)
        ## YOUR CODE HERE
        error = (left_mistakes + right_mistakes) / num_data_points

        # If this is the best error we have found so far, store the feature as best_feature and the error as best_error
        ## YOUR CODE HERE
        if error < best_error:
            best_feature = feature            
            best_error = error              
    
    return best_feature # Return the best feature we found

To test your `best_splitting_feature` function, run the following code:

In [32]:
if best_splitting_feature(train_data, features, 'safe_loans') == 'term. 36 months':
    print('Test passed!')
else:
    print('Test failed... try again!')

Test failed... try again!


## Building the tree

With the above functions implemented correctly, we are now ready to build our decision tree. Each node in the decision tree is represented as a dictionary which contains the following keys and possible values:

    { 
       'is_leaf'            : True/False.
       'prediction'         : Prediction at the leaf node.
       'left'               : (dictionary corresponding to the left tree).
       'right'              : (dictionary corresponding to the right tree).
       'splitting_feature'  : The feature that this node splits on.
    }

First, we will write a function that creates a leaf node given a set of target values. Fill in the places where you find `## YOUR CODE HERE`. There are **three** places in this function for you to fill in.

In [33]:
def create_leaf(target_values):
    
    # Create a leaf node
    leaf = {'splitting_feature' : None,
            'left' : None,
            'right' : None,
            'is_leaf':  True  }   ## YOUR CODE HERE
    
    # Count the number of data points that are +1 and -1 in this node.
    num_ones = len(target_values[target_values == +1])
    num_minus_ones = len(target_values[target_values == -1])
    
    # For the leaf node, set the prediction to be the majority class.
    # Store the predicted class (1 or -1) in leaf['prediction']
    if num_ones > num_minus_ones:
        leaf['prediction'] = +1
    else:
        leaf['prediction'] =  -1
        
    # Return the leaf node        
    return leaf 

We have provided a function that learns the decision tree recursively and implements 3 stopping conditions:
1. **Stopping condition 1:** All data points in a node are from the same class.
2. **Stopping condition 2:** No more features to split on.
3. **Additional stopping condition:** In addition to the above two stopping conditions covered in lecture, in this assignment we will also consider a stopping condition based on the **max_depth** of the tree. By not letting the tree grow too deep, we will save computational effort in the learning process. 

Now, we will write down the skeleton of the learning algorithm. Fill in the places where you find `## YOUR CODE HERE`. There are **seven** places in this function for you to fill in.

In [35]:
def decision_tree_create(data, features, target, current_depth = 0, max_depth = 10):
    remaining_features = features[:] # Make a copy of the features.
    
    target_values = data[target]
    print("--------------------------------------------------------------------")
    print("Subtree, depth = %s (%s data points)." % (current_depth, len(target_values)))
    

    # Stopping condition 1
    # (Check if there are mistakes at current node.
    # Recall you wrote a function intermediate_node_num_mistakes to compute this.)
    if intermediate_node_num_mistakes(target_values) == 0:  ## YOUR CODE HERE
        print("Stopping condition 1 reached.")   
        # If not mistakes at current node, make current node a leaf node
        return create_leaf(target_values)
    
    # Stopping condition 2 (check if there are remaining features to consider splitting on)
    if remaining_features == []:   ## YOUR CODE HERE
        print("Stopping condition 2 reached.")
        # If there are no remaining features to consider, make current node a leaf node
        return create_leaf(target_values)    
    
    # Additional stopping condition (limit tree depth)
    if current_depth >= max_depth:  ## YOUR CODE HERE
        print("Reached maximum depth. Stopping for now.")
        # If the max tree depth has been reached, make current node a leaf node
        return create_leaf(target_values)

    # Find the best splitting feature (recall the function best_splitting_feature implemented above)
    ## YOUR CODE HERE
    splitting_feature = best_splitting_feature(data, features, target)
    
    # Split on the best feature that we found. 
    left_split = data[data[splitting_feature] == 0]
    right_split = data[data[splitting_feature] == 1]
    remaining_features.remove(splitting_feature)
    print("Split on feature %s. (%s, %s)" % (\
                      splitting_feature, len(left_split), len(right_split)))
    
    # Create a leaf node if the split is "perfect"
    if len(left_split) == len(data):
        print("Creating leaf node.")
        return create_leaf(left_split[target])
    if len(right_split) == len(data):
        print("Creating leaf node.")
        return create_leaf(right_split[target])

        
    # Repeat (recurse) on left and right subtrees
    left_tree = decision_tree_create(left_split, remaining_features, target, current_depth + 1, max_depth)        
    ## YOUR CODE HERE
    right_tree = decision_tree_create(right_split, remaining_features, target, current_depth + 1, max_depth) 

    return {'is_leaf'          : False, 
            'prediction'       : None,
            'splitting_feature': splitting_feature,
            'left'             : left_tree, 
            'right'            : right_tree}

Here is a recursive function to count the nodes in your tree:

In [36]:
def count_nodes(tree):
    if tree['is_leaf']:
        return 1
    return 1 + count_nodes(tree['left']) + count_nodes(tree['right'])

Run the following test code to check your implementation. Make sure you get **'Test passed'** before proceeding.

In [38]:
small_data_decision_tree = decision_tree_create(train_data, features, 'safe_loans', max_depth = 3)
if count_nodes(small_data_decision_tree) == 13:
    print('Test passed!')
else:
    print('Test failed... try again!')
    print ('Number of nodes found                :', count_nodes(small_data_decision_tree))
    print ('Number of nodes that should be there : 13' )

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Split on feature grade. (0, 0)
--------------------------------------------------------------------
Subtree, depth = 1 (0 data points).
Stopping condition 1 reached.
--------------------------------------------------------------------
Subtree, depth = 1 (0 data points).
Stopping condition 1 reached.
Test failed... try again!
Number of nodes found                : 3
Number of nodes that should be there : 13


In [39]:
small_data_decision_tree

{'is_leaf': False,
 'prediction': None,
 'splitting_feature': 'grade',
 'left': {'splitting_feature': None,
  'left': None,
  'right': None,
  'is_leaf': True,
  'prediction': -1},
 'right': {'splitting_feature': None,
  'left': None,
  'right': None,
  'is_leaf': True,
  'prediction': -1}}

## Build the tree!

Now that all the tests are passing, we will train a tree model on the **train_data**. Limit the depth to 6 (**max_depth = 6**) to make sure the algorithm doesn't run for too long. Call this tree **my_decision_tree**. 

**Warning**: This code block may take 1-2 minutes to learn. 

In [40]:
# Make sure to cap the depth at 6 by using max_depth = 6
my_decision_tree = decision_tree_create(train_data, features, 'safe_loans', max_depth = 6)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Split on feature grade. (0, 0)
--------------------------------------------------------------------
Subtree, depth = 1 (0 data points).
Stopping condition 1 reached.
--------------------------------------------------------------------
Subtree, depth = 1 (0 data points).
Stopping condition 1 reached.


In [41]:
my_decision_tree

{'is_leaf': False,
 'prediction': None,
 'splitting_feature': 'grade',
 'left': {'splitting_feature': None,
  'left': None,
  'right': None,
  'is_leaf': True,
  'prediction': -1},
 'right': {'splitting_feature': None,
  'left': None,
  'right': None,
  'is_leaf': True,
  'prediction': -1}}

## Making predictions with a decision tree

As discussed in the lecture, we can make predictions from the decision tree with a simple recursive function. Below, we call this function `classify`, which takes in a learned `tree` and a test point `x` to classify.  We include an option `annotate` that describes the prediction path when set to `True`.

Fill in the places where you find `## YOUR CODE HERE`. There is **one** place in this function for you to fill in.

In [43]:
def classify(tree, x, annotate = False):   
    # if the node is a leaf node.
    if tree['is_leaf']:
        if annotate: 
            print("At leaf, predicting %s" % tree['prediction'])
        return tree['prediction'] 
    else:
        # split on feature.
        split_feature_value = x[tree['splitting_feature']]
        if annotate: 
            print("Split on %s = %s" % (tree['splitting_feature'], split_feature_value))
        if split_feature_value == 0:
            return classify(tree['left'], x, annotate)
        else:
            return classify(tree['right'], x, annotate)

Now, let's consider the first example of the test set and see what `my_decision_tree` model predicts for this data point.

In [44]:
test_data[0]

{'grade': 'D',
 'term': '60 months',
 'home_ownership': 'RENT',
 'emp_length': '2 years',
 'safe_loans': -1}

In [46]:
print ('Predicted class: %s ' % classify(my_decision_tree, test_data[0]))

Predicted class: -1 


Let's add some annotations to our prediction to see what the prediction path was that lead to this predicted class:

In [47]:
classify(my_decision_tree, test_data[0], annotate=True)

Split on grade = D
At leaf, predicting -1


-1

In [48]:
classify(small_data_decision_tree, test_data[0], annotate=True)

Split on grade = D
At leaf, predicting -1


-1

** Quiz question:** What was the feature that **my_decision_tree** first split on while making the prediction for test_data[0]?

** Quiz question:** What was the first feature that lead to a right split of test_data[0]?

** Quiz question:** What was the last feature split on before reaching a leaf node for test_data[0]?

## Evaluating your decision tree

Now, we will write a function to evaluate a decision tree by computing the classification error of the tree on the given dataset.

Again, recall that the **classification error** is defined as follows:
$$
\mbox{classification error} = \frac{\mbox{# mistakes}}{\mbox{# total examples}}
$$

Now, write a function called `evaluate_classification_error` that takes in as input:
1. `tree` (as described above)
2. `data` (an SFrame)

This function should return a prediction (class label) for each row in `data` using the decision `tree`. Fill in the places where you find `## YOUR CODE HERE`. There is **one** place in this function for you to fill in.

In [49]:
def evaluate_classification_error(tree, data):
    # Apply the classify(tree, x) to each row in your data
    prediction = data.apply(lambda x: classify(tree, x))    
    # Once you've made the predictions, calculate the classification error and return it
    ## YOUR CODE HERE
    num_of_mistakes = (prediction != data[target]).sum()/float(len(data))
    return num_of_mistakes

Now, let's use this function to evaluate the classification error on the test set.

In [50]:
evaluate_classification_error(my_decision_tree, test_data)

0.496553209823352